# [Day 1: Report Repair](https://adventofcode.com/2020/day/1)

## Part 1

Given an expends report with an expense (integer) on each line, find two expenses that sum to 2020 and return their product.

Example:

`1721`<br>
`979`<br>
`366`<br>
`299`<br>
`675`<br>
`1456`

`1721 + 299 = 2020,`so return`514579`.

## Part 2

Find three expenses that satisfy the criteria above.

In [517]:
from typing import Optional, Set

def day1_part1(expenses: Set[int]) -> Optional[int]:
    for expense in expenses:
        if 2020 - expense in expenses:
            return expense * (2020 - expense)
    return None

def day1_part2(expenses: Set[int]) -> Optional[int]:
    for expense in expenses:
        for expense2 in expenses:
            if expense == expense2:
                continue
            expense3 = 2020 - (expense + expense2)
            if expense3 in expenses:
                return expense * expense2 * expense3
    return None

expense_report = frozenset(map(int, open('input/day1.txt', 'r').readlines()))
print('Part 1:', day1_part1(expense_report))
print('Part 2:', day1_part2(expense_report))

Part 1: 972576
Part 2: 199300880


# [Day 2: Password Philosophy](https://adventofcode.com/2020/day/2)

## Part 1

Given a list of password policies and passwords, return how many passwords are valid.

Format: `[min #]-[max #] [required letter]: [password]`

Example: `1-3 a: abcde`

In the example, the password must contain at least one and no more than three 'a's.

## Part 2

The numbers in the policy now refer to two 1-indexed positions in the password, exactly one of which must be the required letter.

In [518]:
from dataclasses import dataclass
from functools import reduce
import re
from typing import Tuple, List

@dataclass
class PasswordAndPolicy:
    range: Tuple[int, int]
    letter: str
    password: str

def parse_password(line: str) -> PasswordAndPolicy:
    groups = re.match('(\d+)-(\d+) ([a-z]): ([a-z]+)', line).groups()
    return PasswordAndPolicy((int(groups[0]), int(groups[1])), groups[2], groups[3])

def day2_part1(passwords: List[PasswordAndPolicy]) -> int:
    valid = 0
    for p in passwords:
        if p.range[0] <= reduce(lambda count, char: count + 1 if char == p.letter else count, p.password, 0) <= p.range[1]:
            valid += 1
    return valid

def day2_part2(passwords: List[PasswordAndPolicy]) -> int:
    valid = 0
    for p in passwords:
        if (p.password[p.range[0] - 1] == p.letter) ^ (p.password[p.range[1] - 1] == p.letter):
            valid += 1
    return valid

password_list = list(map(parse_password, open('input/day2.txt', 'r').readlines()))
print('Part 1:', day2_part1(password_list))
print('Part 2:', day2_part2(password_list))

Part 1: 467
Part 2: 441


# [Day 3: Toboggan Trajectory](https://adventofcode.com/2020/day/3)

## Part 1

Given a map of open squares (`.`) and trees (`#`), start at the top left, travel 3 characters to the right and 1 character down each step,
and count how many trees you hit until getting to the bottom row. The map repeats horizontally as many times as necessary.

## Part 2

Part 1 used a slope of $-1/3$. Try $-1$, $-1/3$, $-1/5$, $-1/7$, and $-2$ and multiply the results together.

In [519]:
def traverse(map_lines: List[str], right: int, down: int) -> int:
    trees = 0
    for index, line in enumerate(map_lines[::down]):
        if line[(index * right) % len(line)] == '#':
            trees += 1
    return trees

def day3_part1(map_lines: List[str]) -> int:
    return traverse(map_lines, 3, 1)

def day3_part2(map_lines: List[str]) -> int:
    return reduce(lambda x, y: x * y, map(lambda r_d: traverse(map_lines, r_d[0], r_d[1]), [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]))

map_input = open('input/day3.txt', 'r').read().splitlines()
print('Part 1:', day3_part1(map_input))
print('Part 2:', day3_part2(map_input))

Part 1: 153
Part 2: 2421944712


# [Day 4: Passport Processing](https://adventofcode.com/2020/day/4)

## Part 1

Valid passports have the following fields: `byr, iyr, eyr, hgt, hcl, ecl, pid, cid`. `cid` can be missing, but nothing else can.

Given a file with passports separated by empty lines, return the number of valid passports.

## Part 2

Field validation rules:

`byr: [1920, 2002]`<br>
`iyr: [2010, 2020]`<br>
`eyr: [2020, 2030]`<br>
`hgt: [150, 193]cm or [59, 76]in`<br>
`hcl: #[0-9]|[a-f]`<br>
`ecl: amb|blu|brn|gry|grn|hzl|oth`<br>
`pid: 9 digit number including leading zeroes`<br>
`cid: ignored`

In [520]:
@dataclass
class Passport:
    byr: int = None
    iyr: int = None
    eyr: int = None
    hgt: str = None
    hcl: str = None
    ecl: str = None
    pid: str = None

def parse_passport(passport: str) -> Passport:
    fields = dict(map(lambda x: x.split(':'), passport.split()))
    return Passport(int(fields.get('byr') or 0), int(fields.get('iyr') or 0), int(fields.get('eyr') or 0), fields.get('hgt') or "", fields.get('hcl') or "", fields.get('ecl') or "", fields.get('pid') or "")

def day4_part1(passports: List[Passport]) -> int:
    return sum(map(lambda p: all([p.byr, p.iyr, p.eyr, p.hgt, p.hcl, p.ecl, p.pid]), passports))

def day4_part2(passports: List[Passport]) -> int:
    def is_valid(passport: Passport) -> bool:
        height_re = re.match('(\d*)(cm|in)', passport.hgt)
        return all([passport.byr in range(1920, 2003),
                    passport.iyr in range(2010, 2021),
                    passport.eyr in range(2020, 2031),
                    height_re and ((height_re.group(2) == 'cm' and int(height_re.group(1)) in range(150, 194)) or (height_re.group(2) == 'in' and int(height_re.group(1)) in range(59, 77))),
                    re.match('#[0-9|a-f]{6}', passport.hcl),
                    re.match('amb|blu|brn|gry|grn|hzl|oth', passport.ecl),
                    re.match('\d{9}$', passport.pid)])
    return sum(map(is_valid, passports))

passport_list = list(map(parse_passport, open('input/day4.txt', 'r').read().split('\n\n')))
print('Part 1:', day4_part1(passport_list))
print('Part 2:', day4_part2(passport_list))

Part 1: 264
Part 2: 224


# [Day 5: Binary Boarding](https://adventofcode.com/2020/day/5)

## Part 1

Seats are numbered in binary partitioning, example `FBFBBFFRLR`. `F` and `B` are front and back half of rows, starting with
the range 0-127. `L` and `R` are left and right half of columns, starting with 0-7.

Seat ID is the row multiplied by 8 plus the column. Return the highest seat ID in the input.

## Part 2

Return the only missing seat ID, whose neighboring seats are in the input

In [521]:
def parse_seat(boarding_pass: str) -> Tuple[int, int]:
    return int(boarding_pass[:7].replace('F', '0').replace('B', '1'), 2), int(boarding_pass[7:].replace('L', '0').replace('R', '1'), 2)

def seat_id(seat: Tuple[int, int]) -> int:
    return seat[0] * 8 + seat[1]

def day5_part1(boarding_passes: List[str]) -> int:
    return max(map(seat_id, map(parse_seat, boarding_passes)))

def day5_part2(boarding_passes: List[str]) -> int:
    seats = sorted(map(seat_id, map(parse_seat, boarding_passes)))
    previous = seats[0] - 1
    for seat in seats:
        if seat - 1 != previous:
            return seat - 1
        previous = seat
    return -1

boarding_pass_list = open('input/day5.txt', 'r').readlines()
print('Part 1:', day5_part1(boarding_pass_list))
print('Part 2:', day5_part2(boarding_pass_list))

Part 1: 991
Part 2: 534


# [Day 6: Custom Customs](https://adventofcode.com/2020/day/6)

## Part 1

Given groups of lines with letters indicating questions someone in the group answered "yes" to, return the sum of such questions
for all groups.

## Part 2

Return how many questions everyone in the group answered "yes" to.

In [522]:
def day6_part1(declarations: List[str]) -> int:
    return sum(map(lambda d: len(set(d.replace('\n', ''))), declarations))

def day6_part2(declarations: List[str]) -> int:
    return sum(map(lambda d: len(set.intersection(*map(set, d.split('\n')))), declarations))

customs_list = open('input/day6.txt', 'r').read().split('\n\n')
print('Part 1:', day6_part1(customs_list))
print('Part 2:', day6_part2(customs_list))

Part 1: 6310
Part 2: 3193


# [Day 7: Handy Haversacks](https://adventofcode.com/2020/day/7)

## Part 1

Given rules about what bags can be inside other bags, return how many bags can eventually hold a shiny gold bag.

## Part 2

Return how many bags are inside a shiny gold bag.

In [523]:
from typing import Dict

Rules = Dict[str, List[Tuple[int, str]]]

def parse_rules(rules_list: List[str]) -> Rules:
    rules = {}
    for rule in rules_list:
        outer_bag, rest = re.match('(.*) bags contain (.*)', rule).groups()
        inner_bags = []
        if rest != 'no other bags.':
            inner_bags = list(map(lambda x: (int(x[0]), x[2:x.find(' bag')]), rest.split(', ')))
        rules[outer_bag] = inner_bags
    return rules

def day7_part1(rules: Rules) -> int:
    def check_bag(bag: str) -> bool:
        inner_bags = list(map(lambda x: x[1], rules[bag]))
        if 'shiny gold' in inner_bags:
            return True
        return any(map(check_bag, inner_bags))
    return sum(map(check_bag, rules.keys()))

def day7_part2(rules: Rules) -> int:
    def count_bag(bag: str) -> int:
        return sum(map(lambda x: x[0], rules[bag])) + sum(map(lambda x: x[0] * count_bag(x[1]), rules[bag]))
    return count_bag('shiny gold')

bag_rules = parse_rules(open('input/day7.txt', 'r').readlines())
print('Part 1:', day7_part1(bag_rules))
print('Part 2:', day7_part2(bag_rules))

Part 1: 272
Part 2: 172246


# [Day 8: Handheld Halting](https://adventofcode.com/2020/day/8)

## Part 1

Run the given bytecode, and return the value of the accumulator just before any instruction is executed for a second time.

## Part 2

Replace a `jmp` with a `nop` or vice versa to make the program terminate by running the instruction one beyond the end of the file,
and return the accumulator after the program ends.

In [524]:
Code = List[Tuple[str, int]]

def execute(code: Code) -> (int, bool):
    accumulator = 0
    executed = set()
    i = 0
    while i < len(code):
        if i in executed:
            break
        executed.add(i)
        operation, argument = code[i]
        if operation == 'jmp':
            i += argument
            continue
        elif operation == 'acc':
            accumulator += argument
        i += 1
    return accumulator, i == len(code)

def day8_part1(code: Code) -> int:
    return execute(code)[0]

def day8_part2(code: Code) -> int:
    for i in range(len(code)):
        if code[i][0] == 'nop' or code[i][0] == 'jmp':
            new_code = code.copy()
            new_code[i] = ('nop' if code[i][0] == 'jmp' else 'jmp', code[i][1])
            accumulator, terminated = execute(new_code)
            if terminated:
                return accumulator

boot_code = list(map(lambda i: (i[0], int(i[1])), map(lambda x: x.split(), open('input/day8.txt', 'r').readlines())))
print('Part 1:', day8_part1(boot_code))
print('Part 2:', day8_part2(boot_code))

Part 1: 1727
Part 2: 552


# [Day 9: Encoding Error](https://adventofcode.com/2020/day/9)

## Part 1

Each number in the input must be the sum of two different numbers in the previous 25. Find the first number which fails this test.

## Part 2

Find a contiguous subset that sums to the answer to part 1, and return the sum of the smallest and largest numbers in the subset.

In [525]:
import collections

def day9_part1(numbers: List[int]) -> int:
    sums = collections.deque()
    for i in range(25):
        sums.append(list(map(lambda x: x + numbers[i], numbers[:i] + numbers[i + 1:25])))
    for i in range(25, len(numbers)):
        if not any(list(map(lambda x: numbers[i] in x, sums))):
            return numbers[i]
        sums.popleft()
        sums.append(list(map(lambda x: x + numbers[i], numbers[i - 25:i])))
    return -1

def day9_part2(numbers: List[int]) -> int:
    target = day9_part1(numbers)
    for i in range(len(numbers)):
        for j in range(i, len(numbers)):
            total = sum(numbers[i:j])
            if total == target:
                return min(numbers[i:j]) + max(numbers[i:j])
            if total > target:
                break
    return -1

number_list = list(map(int, open('input/day9.txt', 'r').readlines()))
print('Part 1:', day9_part1(number_list))
print('Part 1:', day9_part2(number_list))

Part 1: 29221323
Part 1: 4389369


# [Day 10: Adapter Array](https://adventofcode.com/2020/day/10)

## Part 1

In a chain using all given adapters, return the number of gaps of 1 multiplied by the number of gaps of 3.

## Part 2

Return how many ways the adapters can be arranged to connect the wall and the device.

In [526]:
def day10_part1(adapters: List[int]) -> int:
    chain = [0] + sorted(adapters) + [max(adapters) + 3]
    one, three = 0, 0
    for i in range(1, len(chain)):
        difference = chain[i] - chain[i - 1]
        if difference == 1:
            one += 1
        if difference == 3:
            three += 1
    return one * three

def day10_part2(adapters: List[int]) -> int:
    sorted_adapters = [0] + sorted(adapters)
    runs = [1]
    for i in range(1, len(sorted_adapters)):
        if sorted_adapters[i] - sorted_adapters[i - 1] == 1:
            runs[-1] += 1
        else:
            runs.append(1)
    total = 1
    for run in runs:
        if run == 3:
            total *= 2
        elif run == 4:
            total *= 4
        elif run == 5:
            total *= 7
    return total

adapter_list = list(map(int, open('input/day10.txt', 'r').readlines()))
print('Part 1:', day10_part1(adapter_list))
print('Part 2:', day10_part2(adapter_list))

Part 1: 2590
Part 2: 226775649501184


# [Day 11: Seating System](https://adventofcode.com/2020/day/11)

## Part 1

Given a grid where `.` is floor, `L` is an empty seat, and `#` is an occupied seat, simulate rounds where an empty seat
with no occupied neighbors becomes occupied, and an occupied seat with four or more occupied neighbors becomes empty.

Return how many occupied seats there are after the simulation stabilizes.

## Part 2

Consider first seat in each direction rather than just adjacent seats, occupied seats now empty with five other visible occupied seats.

In [528]:
from typing import Callable
import copy

def run_simulation(seat_map: List[List[str]], too_crowded: int, change_occupancy: Callable[[List[List[str]], List[List[int]], int, int, int], None]) -> int:
    adjacency_map = [[0] * len(seat_map[0]) for _ in range(len(seat_map))]
    previously_occupied, occupied_seats = -1, 0
    while previously_occupied != occupied_seats:
        previously_occupied = occupied_seats
        previous_adjacency_map = copy.deepcopy(adjacency_map)
        for i in range(len(seat_map)):
            for j in range(len(seat_map[0])):
                if seat_map[i][j] == 'L' and previous_adjacency_map[i][j] == 0:
                    seat_map[i][j] = '#'
                    change_occupancy(seat_map, adjacency_map, i, j, 1)
                    occupied_seats += 1
                elif seat_map[i][j] == '#' and previous_adjacency_map[i][j] >= too_crowded:
                    seat_map[i][j] = 'L'
                    change_occupancy(seat_map, adjacency_map, i, j, -1)
                    occupied_seats -= 1
    return occupied_seats

def day11_part1(original_seat_map: List[List[str]]) -> int:
    def change_occupancy(seat_map: List[List[str]], adjacency_map: List[List[int]], row: int, column: int, change: int):
        for r in range(max(0, row - 1), min(len(seat_map), row + 2)):
            for c in range(max(0, column - 1), min(len(seat_map[0]), column + 2)):
                if not (r == row and c == column):
                    adjacency_map[r][c] += change
    return run_simulation(copy.deepcopy(original_seat_map), 4, change_occupancy)

def day11_part2(original_seat_map: List[List[str]]) -> int:
    def change_occupancy(seat_map: List[List[str]], adjacency_map: List[List[int]], row: int, column: int, change: int):
        def first_visible_seat(row_change: int, column_change: int) -> Optional[Tuple[int, int]] :
            r, c = row + row_change, column + column_change
            while 0 <= r < len(seat_map) and 0 <= c < len(seat_map[0]):
                if seat_map[r][c] in 'L#':
                    return r, c
                r += row_change
                c += column_change
            return None
        visible_seats = filter(
            lambda x: x is not None,
            map(lambda direction: first_visible_seat(*direction), [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)])
        )
        for seat in visible_seats:
            adjacency_map[seat[0]][seat[1]] += change
    return run_simulation(copy.deepcopy(original_seat_map), 5, change_occupancy)

seat_map_input = list(map(list, open('input/day11.txt', 'r').readlines()))
print('Part 1:', day11_part1(seat_map_input))
print('Part 2:', day11_part2(seat_map_input))

Part 1: 2113
Part 2: 1865
